#### 05 — RAG Logging (Observability Layer)

This notebook defines and validates the logging schema for the RAG system.
It creates the Delta table, verifies the schema, and demonstrates example queries.

All reusable logging functions (e.g., `log_rag_event`) live in `00_utils.ipynb`.

In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

### Design notes

This table captures full RAG observability:

- What was asked (question)
- What was retrieved (retrieved_chunks)
- What was generated (answer)
- Which models were used (deployments)
- When it happened (created_at)

This enables:
- Debugging hallucinations
- Reproducing failures
- Offline evaluation
- Quality monitoring
- A/B testing

The logging function is intentionally separated into `00_utils.ipynb` so it can be reused by:
- Batch pipelines
- APIs
- Streaming endpoints
- UI demos

In [0]:
# Create the query log table (Unity Catalog)

ensure_rag_log_table(spark, RAG_LOG_TABLE)

✅ RAG log table ensured: databricks_rag_demo.default.rag_query_logs


In [0]:
spark.table(RAG_LOG_TABLE).printSchema()

root
 |-- query_id: string (nullable = true)
 |-- question: string (nullable = true)
 |-- top_k: integer (nullable = true)
 |-- retriever_type: string (nullable = true)
 |-- retrieved_chunks: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- chunk_id: string (nullable = true)
 |    |    |-- doc_id: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- chunk_index: integer (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- score: double (nullable = true)
 |-- prompt: string (nullable = true)
 |-- answer: string (nullable = true)
 |-- embedding_deployment: string (nullable = true)
 |-- chat_deployment: string (nullable = true)
 |-- created_at: timestamp (nullable = true)



In [0]:
# Inspect recent logs

spark.sql(f"""
SELECT
  created_at,
  query_id,
  question,
  retriever_type,
  top_k,
  retrieved_chunks[0].url AS top_source
FROM {RAG_LOG_TABLE}
ORDER BY created_at DESC
LIMIT 10
""").display()

created_at,query_id,question,retriever_type,top_k,top_source
2026-01-16T06:49:38.556206Z,c1d014f3-80a0-480d-9e67-75c1c60b9539,What is Azure VM Scale Sets?,B,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/flexible-virtual-machine-scale-sets-powershell.md
2026-01-16T06:49:30.934227Z,534688cd-03c9-4729-8837-e38e27866f33,What is Azure VM Scale Sets?,A,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/flexible-virtual-machine-scale-sets-powershell.md
2026-01-16T06:49:21.01436Z,f6d7f27f-f630-4ea1-a50f-46997963d162,How do I resize an Azure virtual machine?,B,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-customization.md
2026-01-16T06:49:12.399182Z,7f89aade-6341-412c-8289-320fa7ec0a3b,How do I resize an Azure virtual machine?,A,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-customization.md
2026-01-16T06:49:02.860406Z,8d759cd9-f704-4e63-ade5-944c33af3ba5,What is the difference between a normal Azure VM and an ephemeral VM?,B,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-16T06:48:52.963109Z,e9f6e839-c5ad-415b-9cf0-6ad44ecc2a0f,What is the difference between a normal Azure VM and an ephemeral VM?,A,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md


In [0]:
# Only use during development for cleanup

# spark.sql(f"""
# TRUNCATE TABLE {RAG_LOG_TABLE}
# """)